<a href="https://colab.research.google.com/github/mobarakol/Project_Gen/blob/main/SurgicalGPT%2B%2B_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downlaod code and dataset

In [1]:
#Download code
!git clone https://github.com/mobarakol/Project_Gen.git
!mkdir /content/Project_Gen/datasets
%cd /content/Project_Gen/datasets

#Download Dataset
!gdown --id 1WGdztykX3nW6pi_BKp4rO8nA7ESNRfVN

# Unzipping the VQA EndoVis18 Dataset\
!unzip -q EndoVis-18-VQA.zip

%cd /content/Project_Gen

Cloning into 'Project_Gen'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 61 (delta 29), reused 25 (delta 8), pack-reused 0
Receiving objects: 100% (61/61), 28.77 KiB | 516.00 KiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/Project_Gen/datasets
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1WGdztykX3nW6pi_BKp4rO8nA7ESNRfVN
To: /content/Project_Gen/datasets/EndoVis-18-VQA.zip
100% 2.70G/2.70G [00:49<00:00, 54.9MB/s]
/content/Project_Gen


Installing requirements

In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.8 MB/s eta 0:00:00


Running Script:<br>
%cd /content/Project_Gen
import os
os.makedirs('checkpoints/efvlegpt2rs18', exist_ok=True)
!python train_SGPT_V2_Sentence.py \
--lr=0.00001 \
--checkpoint_dir='checkpoints/efvlegpt2rs18/m18_v1_z_qf_' \
--dataset_type='m18' \
--tokenizer_ver='gpt2v1' \
--model_ver='efvlegpt2rs18' \
--model_subver='v1' \
--vis_pos_emb='zeroes'\
--batch_size=40

In [ ]:
%cd /content/Project_Gen
import os
import sys
import argparse
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch import optim
from torch.nn import CrossEntropyLoss
from torch.utils.data  import DataLoader
from transformers import GPT2Tokenizer
import torch.backends.cudnn as cudnn
# from model.EFGPT2Sentence import EFVLEGPT2RS18Sentence
from dataloader.dataloaderGPT2Sentence import EndoVis18VQAGPTSentence
from utils import AverageMeter, save_clf_checkpoint, adjust_learning_rate

import torch.nn.functional as F
from torchvision import models
from transformers import  VisualBertConfig, GPT2Config
from transformers import VisualBertModel, GPT2Model, ViTModel, SwinModel
from transformers import GPT2LMHeadModel
from nltk.translate.bleu_score import corpus_bleu

def seed_everything(seed=27):
    '''
    Set random seed for reproducible experiments
    Inputs: seed number
    '''
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class EFVLEGPT2RS18Sentence(nn.Module):
    def __init__(self, model_subver = 'v3', tokenizer_len=50258, vis_pos_emb = None):
        super(EFVLEGPT2RS18Sentence, self).__init__()
        '''
        v0: visual embedding : Default patch1 + embedding form VB + GPT2 decoder
        v1: visual embedding : Default patch1 + from nn.linear    + GPT2 decoder
        v2: visual embedding : visual patches + embedding form VB + GPT2 decoder
        v3: visual embedding : visual patches + from nn.linear    + GPT2 decoder
        '''

        self.sub_ver = model_subver
        self.vis_pos_emb = vis_pos_emb

        ## image processing
        self.img_feature_extractor = models.resnet18(pretrained=True)
        if self.sub_ver == 'v0' or self.sub_ver =='v1':
            new_fc = nn.Sequential(*list(self.img_feature_extractor.fc.children())[:-1])
            self.img_feature_extractor.fc = new_fc
        elif self.sub_ver == 'v2' or self.sub_ver =='v3':
            self.img_feature_extractor = torch.nn.Sequential(*(list(self.img_feature_extractor.children())[:-2]))

        ## Visual_embedding
        if self.sub_ver == 'v0' or self.sub_ver =='v2':
            # visual bert embedding
            VB_config = VisualBertConfig.from_pretrained("uclanlp/visualbert-vqa-coco-pre")
            VB_config.visual_embedding_dim = 512
            visualbert = VisualBertModel(config=VB_config)
            self.visual_embedder = visualbert.embeddings.visual_projection
        elif self.sub_ver == 'v1' or self.sub_ver =='v3':
            self.visual_embedder = nn.Linear(512, 768)

        ## word_embedding default GPT2 word embedding
        gpt2configuration = GPT2Config()
        word_embedder = GPT2Model(gpt2configuration)
        word_embedder.resize_token_embeddings(tokenizer_len)
        self.word_embedder = word_embedder.wte

        ## GPT2 visual context aware decoder
        self.VCAdecoder = GPT2LMHeadModel.from_pretrained('gpt2')


    def forward(self, question, img, answer):

        ## image encoder features
        img_feature = self.img_feature_extractor(img)

        if self.sub_ver == 'v0' or self.sub_ver =='v1':
            img_feature = torch.unsqueeze(img_feature, dim=1)
        if self.sub_ver == 'v2'or self.sub_ver =='v3':
            img_feature = torch.flatten(img_feature, start_dim=2)
            img_feature = img_feature.permute((0,2,1))


        ## visual Embedding : id type 1, pos: zero / incremental
        visual_embeds = self.visual_embedder(img_feature)
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float)
        visual_attention_mask = visual_attention_mask.to(device)

        ## question embedding:
        question['input_ids'] = question['input_ids'].to(device)
        question_embeds = self.word_embedder(question['input_ids'])
        question_attention_mask = question['attention_mask'].to(device)

        ## answer embedding
        answer['input_ids'] = answer['input_ids'].to(device)
        answer_embeds = self.word_embedder(answer['input_ids'])
        answer_attention_mask = answer['attention_mask'].to(device)

        ## token type and position id for question
        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            question_id_type = torch.zeros(*question_embeds.size()[:-1], dtype=torch.long, device=device)
            question_position_id = torch.arange(0,question_embeds.size()[1])
            question_position_id = torch.unsqueeze(question_position_id,0)
            question_position_id = question_position_id.repeat(question_embeds.size()[0], 1)
            question_position_id = question_position_id.to(device)
            question_len = len(question_position_id[0])

        ## token type and position id for vision
        if self.vis_pos_emb == 'zeroes':
            visual_id_type = torch.ones(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
            visual_position_id = torch.zeros(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
        elif self.vis_pos_emb == 'pos':
            visual_id_type = torch.ones(*visual_embeds.size()[:-1], dtype=torch.long, device=device)
            visual_position_id = torch.arange(0,visual_embeds.size()[1])
            visual_position_id = torch.unsqueeze(visual_position_id,0)
            visual_position_id = visual_position_id.repeat(visual_embeds.size()[0], 1)
            visual_position_id += (question_len)
            visual_position_id = visual_position_id.to(device)
        visual_len = len(visual_position_id[0])

        ## token type and position id for answer
        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            answer_id_type = torch.zeros(*answer_embeds.size()[:-1], dtype=torch.long, device=device)
            answer_position_id = torch.arange(0,answer_embeds.size()[1])
            answer_position_id = torch.unsqueeze(answer_position_id,0)
            answer_position_id = answer_position_id.repeat(answer_embeds.size()[0], 1)
            answer_position_id += (question_len+visual_len)
            answer_position_id = answer_position_id.to(device)


        ## question first
        inputs_embeds = torch.cat((question_embeds, visual_embeds, answer_embeds), dim=1)
        attention_mask = torch.cat((question_attention_mask, visual_attention_mask, answer_attention_mask), dim=1)

        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            token_type_ids = torch.cat((question_id_type, visual_id_type, answer_id_type), dim=1)
            position_ids = torch.cat((question_position_id, visual_position_id, answer_position_id), dim=1)


        ## VCA_GPT2 decoder
        if self.vis_pos_emb == 'zeroes' or self.vis_pos_emb == 'pos':
            out = self.VCAdecoder(inputs_embeds=inputs_embeds, attention_mask=attention_mask, position_ids = position_ids, token_type_ids = token_type_ids)
        else:
            out = self.VCAdecoder(inputs_embeds=inputs_embeds, attention_mask=attention_mask)


        return out

def train(args, train_dataloader, model, criterion, optimizer, epoch, tokenizer, device):

    model.train()

    total_loss = AverageMeter()

    for i, (_, visual_features, questions, answers) in enumerate(tqdm(train_dataloader),0):

        # prepare questions and answers
        question_list = []
        answer_list = []
        for question in questions: question_list.append(question)
        for answer in answers: answer_list.append(answer)

        if args.model_ver == 'efvlegpt2rs18' or args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':
            question_inputs = tokenizer(question_list, padding="max_length",max_length= args.question_len, return_tensors="pt")
            answer_inputs = tokenizer(answer_list, padding="max_length",max_length= args.answer_len, return_tensors="pt")


        # Visual features
        if args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':
            visual_features['pixel_values'] = torch.squeeze(visual_features['pixel_values'],1)
        else:
            visual_features = visual_features.to(device)
            visual_len = 80

        # model forward(question, img, answer)
        logits = model(question_inputs, visual_features, answer_inputs)[0]

        # only consider loss on reference summary just like seq2seq models
        idx = args.question_len + 1

        shift_logits = logits[..., idx:-1, :].contiguous()
        shift_labels = answer_inputs['input_ids'][..., 1:].contiguous() # 1 because answer has '<|sep|>' in front
        shift_labels = shift_labels.to(device)

        loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.update(loss.item())

    print("Epoch: {}/{} Loss: {:.6f} AVG_Loss: {:.6f}".format(epoch, args.epochs, total_loss.val, total_loss.avg))

def validate(args, val_loader, model, criterion, epoch, tokenizer, device, save_output = False):

    references = []
    hypotheses = []

    model.eval()
    total_loss = AverageMeter()

    with torch.no_grad():
        for i, (_, visual_features, questions, answers) in enumerate(tqdm(val_loader),0):

            # prepare questions and answers
            question_list = []
            answer_list = []
            for question in questions: question_list.append(question)
            for answer in answers: answer_list.append(answer)

            if args.model_ver == 'efvlegpt2rs18' or args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':
                question_inputs = tokenizer(question_list, padding="max_length",max_length= args.question_len, return_tensors="pt")
                answer_inputs = tokenizer(answer_list, padding="max_length",max_length= args.answer_len, return_tensors="pt")


            # Visual features
            if args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':
                visual_features['pixel_values'] = torch.squeeze(visual_features['pixel_values'],1)
            else:
                visual_features = visual_features.to(device)
                visual_len = 80

            # model forward(question, img, answer)
            logits = model(question_inputs, visual_features, answer_inputs)[0]


            # only consider loss on reference summary just like seq2seq models
            idx = args.question_len + 1
            shift_logits = logits[..., idx:-1, :].contiguous()
            shift_labels = answer_inputs['input_ids'][..., 1:].contiguous() # 1 because answer has '<|sep|>' in front

            # copy for logits and labels for sentence decoding and blue-4 score calculation
            logits_copy = logits.clone()
            shift_labels_copy = shift_labels.clone()

            # loss calculation
            shift_labels = shift_labels.to(device)
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            total_loss.update(loss.item())

            # references    - Ground truth answer
            answer_GT_dec = tokenizer.batch_decode(shift_labels_copy, skip_special_tokens= True)
            for answer_GT_dec_i in answer_GT_dec: references.append([answer_GT_dec_i.split()])
            # print(references)

            # Hypotheses - predicted answer
            _, answer_Gen_id = torch.max(logits_copy, dim=2)
            answer_Gen_dec = tokenizer.batch_decode(answer_Gen_id, skip_special_tokens= True)
            for answer_Gen_dec_i in answer_Gen_dec: hypotheses.append(answer_Gen_dec_i.split())
            # print(hypotheses)


        # Calculate BLEU1~4
        metrics = {}
        metrics["Bleu_1"] = corpus_bleu(references, hypotheses, weights=(1.00, 0.00, 0.00, 0.00))
        metrics["Bleu_2"] = corpus_bleu(references, hypotheses, weights=(0.50, 0.50, 0.00, 0.00))
        metrics["Bleu_3"] = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0.00))
        metrics["Bleu_4"] = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))

        print("Epoch: {}/{} EVA LOSS: {:.6f} BLEU-1 {:.6f} BLEU2 {:.6f} BLEU3 {:.6f} BLEU-4 {:.6f}".format
          (epoch, args.epochs, total_loss.avg, metrics["Bleu_1"],  metrics["Bleu_2"],  metrics["Bleu_3"],  metrics["Bleu_4"]))

    return metrics

def get_arg():
    parser = argparse.ArgumentParser(description='VisualQuestionAnswerClassification')

    # Training parameters
    parser.add_argument('--epochs',         type=int,   default=80,                                 help='number of epochs to train for (if early stopping is not triggered).') #80, 26
    parser.add_argument('--batch_size',     type=int,   default=50,                                 help='batch_size')
    parser.add_argument('--workers',        type=int,   default=1,                                  help='for data-loading; right now, only 1 works with h5pys.')

    # existing checkpoint
    parser.add_argument('--checkpoint',     default=None,                                           help='path to checkpoint, None if none.')

    parser.add_argument('--lr',             type=float, default=0.00001,                            help=' 0.00001, 0.000005')
    parser.add_argument('--checkpoint_dir', default= 'checkpoints/efvlegpt2rs18/m18/v3_p_qf_',      help='m18/c80')
    parser.add_argument('--dataset_type',   default= 'm18',                                         help='m18/c80')
    parser.add_argument('--tokenizer_ver',  default= 'gpt2v1',                                      help='btv2/btv3/gpt2v1')
    parser.add_argument('--model_subver',   default= 'v3',                                          help='V0,v1/v2/v3/v4')
    parser.add_argument('--question_len',   default= 25,                                            help='25')
    parser.add_argument('--answer_len',     default= 35,                                            help='25')
    parser.add_argument('--model_ver',      default= 'efvlegpt2rs18',                               help='efvlegpt2rs18/efvlegpt2Swin/"')  #vrvb/gpt2rs18/gpt2ViT/gpt2Swin/biogpt2rs18/vilgpt2vqa/efgpt2rs18gr/efvlegpt2Swingr
    parser.add_argument('--vis_pos_emb',    default= 'pos',                                         help='None, zeroes, pos')
    parser.add_argument('--patch_size',     default= 5,                                             help='1/2/3/4/5')

    parser.add_argument('--validate',       default=False,                                          help='When only validation required False/True')

    if 'ipykernel' in sys.modules:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

if __name__ == '__main__':

    seed_everything()

    args = get_arg()
    args.lr = 0.00005
    args.epochs = 80
    args.checkpoint_dir='checkpoints/efvlegpt2rs18/m18_v1_z_qf_'
    args.dataset_type='m18'
    args.tokenizer_ver='gpt2v1'
    args.model_ver='efvlegpt2rs18'
    args.model_subver='v1'
    args.vis_pos_emb='zeroes'
    args.batch_size=40
    os.makedirs('checkpoints/efvlegpt2rs18', exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
    cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead
    print('device =', device)

    # best model initialize
    start_epoch = 1
    best_epoch = [0]
    best_results = [0.0]
    epochs_since_improvement = 0


    if args.dataset_type == 'm18':
        '''
        Train and test dataloader for EndoVis18
        '''
        # tokenizer
        tokenizer = None
        if args.tokenizer_ver == 'gpt2v1':
            tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer_length = len(tokenizer)

        # data location
        train_seq = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
        val_seq = [1, 5, 16]

        folder_head = 'datasets/EndoVis-18-VQA/seq_'
        folder_tail = '/vqa/Sentence/*.txt'

        # dataloader
        if args.model_ver == 'efvlegpt2rs18' or args.model_ver == "efvlegpt2Swin" or args.model_ver == 'efvlegpt2ViT':

            train_dataset = EndoVis18VQAGPTSentence(train_seq, folder_head, folder_tail, model_ver=args.model_ver)
            train_dataloader = DataLoader(dataset=train_dataset, batch_size= args.batch_size, shuffle=True, num_workers=8)
            val_dataset = EndoVis18VQAGPTSentence(val_seq, folder_head, folder_tail, model_ver=args.model_ver)
            val_dataloader = DataLoader(dataset=val_dataset, batch_size= args.batch_size, shuffle=False, num_workers=8)


    # Initialize / load checkpoint
    if args.checkpoint is None:
        if args.model_ver == 'efvlegpt2rs18':
            model = EFVLEGPT2RS18Sentence(model_subver = args.model_subver, tokenizer_len=len(tokenizer), vis_pos_emb = args.vis_pos_emb)
        optimizer = optim.Adam(model.parameters(), lr=args.lr)

    else:
        checkpoint = torch.load(args.checkpoint, map_location=str(device))
        start_epoch = checkpoint['epoch']
        epochs_since_improvement = checkpoint['epochs_since_improvement']
        # best_Acc = checkpoint['Acc']
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
        final_args = checkpoint['final_args']
        for key in final_args.keys(): args.__setattr__(key, final_args[key])


    # Move to GPU, if available
    model = model.to(device)
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print('model params: ', pytorch_total_params)

    # Loss function
    criterion = CrossEntropyLoss().to(device)

    for epoch in range(start_epoch, args.epochs):

        if epochs_since_improvement > 0 and epochs_since_improvement % 5 == 0:
            adjust_learning_rate(optimizer, 0.8)

        # train
        train(args, train_dataloader=train_dataloader, model = model, criterion=criterion, optimizer=optimizer, epoch=epoch, tokenizer = tokenizer, device = device)

        # validation
        metrics = validate(args, val_loader=val_dataloader, model = model, criterion=criterion, epoch=epoch, tokenizer = tokenizer, device = device)

        if metrics["Bleu_4"] >= best_results[0]:
            epochs_since_improvement = 0

            best_results[0] = metrics["Bleu_4"]
            best_epoch[0] = epoch
            save_clf_checkpoint(args.checkpoint_dir, epoch, epochs_since_improvement, model, optimizer, best_results[0])
        else:
            epochs_since_improvement += 1
            print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))




/content/Project_Gen
device = cuda
Total files: 1560 | Total question: 10574
Total files: 447 | Total question: 3216
model params:  174607680


100%|██████████| 265/265 [06:17<00:00,  1.42s/it]


Epoch: 1/80 Loss: 0.104093 AVG_Loss: 1.074687


100%|██████████| 81/81 [01:55<00:00,  1.43s/it]


Epoch: 1/80 EVA LOSS: 0.115519 BLEU-1 0.667146 BLEU2 0.629139 BLEU3 0.592228 BLEU-4 0.541444


...

100%|██████████| 265/265 [06:11<00:00,  1.40s/it]


Epoch: 58/80 Loss: 0.000183 AVG_Loss: 0.000086


100%|██████████| 81/81 [01:55<00:00,  1.42s/it]


Epoch: 58/80 EVA LOSS: 0.153075 BLEU-1 0.745770 BLEU2 0.720656 BLEU3 0.697199 BLEU-4 0.663752

Epochs since last improvement: 14



 54%|█████▍    | 144/265 [03:28<02:22,  1.18s/it]